In [3]:
#download dataset from kaggle in csv format
!pip3 install -i https://test.pypi.org/simple/ supportlib

In [5]:
!pip3 install google.colab

  Could not find a version that satisfies the requirement google.colab (from versions: )
No matching distribution found for google.colab


In [10]:
import wget
wget.download('https://storage.googleapis.com/kaggle-datasets/9914/14036/imdb_master.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1547275066&Signature=oLJbzz5f0%2BHhjZ8MSkCRrq%2FEKsAwpKxfrmF6LnrCz0zWk%2BT3MeLBNOi0t7lO60rC%2FRDCKTvm5H%2F9Sl942ESEb6icE8wJdcF1jgsy6UcB7ulkpnbXDYuWbXYcLyRPFcWSh3a8ZVD4z%2Bl8Eswq5En7h%2FR%2FglLw4DMj3aV42e%2B4pqynt9B1zrTjYDwnI2QmkEV6PYhc2yHmu%2F%2FvAfXGNZQJtFVBf05z%2BFN7nzFElwoqG%2BsqiUl7mELegHKcgckhqvQ8QPrVNwQ%2BYTtjPUU76OzwL4SZqc77maQyybXz%2BYtaYSl%2B0K5wektQWNRdpT1dhEEOw30v6Fz3Rl4EUGjniP7zTA%3D%3D')

'imdb_master.csv.zip'

In [15]:
#unzip data
sl.zipextract('./imdb_master.csv.zip')

In [1]:
#reading imdb_master.csv
import pandas as pd
data = pd.read_csv('./imdb_master.csv',encoding="latin-1")

In [2]:
data = data.sort_values("type")
data = data.drop(['Unnamed: 0','file'],axis=1)
data.columns = ["type","review","sentiment"]
data['sentiment'] = data['sentiment'].map({'pos': 1, 'neg': 0})
data['type'] = data['type'].map({'test':0 , 'train':1})
data = data.dropna()
review = data['review']
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
def cleantext(rev):
  rev = re.sub(r'[^a-zA-Z]',' ',rev)
  rev = rev.lower()
  rev = rev.split()
  rev = [word for word in rev if not word in set(stopwords.words('english'))]
  rev = [lemmatizer.lemmatize(token) for token in rev]
  rev = " ".join(rev)
  return rev

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sanjaymoto75/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/sanjaymoto75/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
review = review.apply(lambda x: cleantext(x))

In [21]:
review.head

<bound method NDFrame.head of 0        mr costner dragged movie far longer necessary ...
16671    aside fact movie filmed mostly rockport beauti...
16670    well done photography sound music performance ...
16669    thought love letter pretty good movie certain ...
16668    film half bad may little long time carried alo...
16667    movie sicky sweet cutesy romantic comedy kind ...
16666    sir played lear time tonight remember opening ...
16665    rita hayworth rusty parker cover girl du jour ...
16664    thought movie pretty good really enjoyed viewe...
16663    love letter one time favorite book naturally s...
16672    one better feel good film kate capshaw leading...
16662    love letter somewhat pleasant low key romantic...
16660    along line comedy error mistaken affection tra...
16659    back bought chinese box set fist north star ca...
16658    watch film like peter watkins privilege story ...
16657    curiosity patience finally see controversial f...
16656    think film made y

In [4]:
data['review'] = review

In [5]:
data.to_csv('cleaned.csv')

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , CuDNNLSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import ReduceLROnPlateau

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['review'])
list_tokenized_train = tokenizer.texts_to_sequences(data['review'])
maxlen = 200
X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
y_train = data['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(CuDNNLSTM(32, return_sequences = True))
model.add(Dropout(0.25))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="elu"))
model.add(Dropout(0.25))
model.add(Dense(1, activation="sigmoid"))

adam = optimizers.Adam(lr=0.01,decay=0.1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

batch_size = 128
epochs = 5
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 10s 249us/step - loss: 0.3368 - acc: 0.8521 - val_loss: 0.3628 - val_acc: 0.8466
Epoch 2/5
40000/40000 [==============================] - 9s 220us/step - loss: 0.2260 - acc: 0.9131 - val_loss: 0.3448 - val_acc: 0.8560
Epoch 3/5
40000/40000 [==============================] - 9s 220us/step - loss: 0.2018 - acc: 0.9238 - val_loss: 0.3416 - val_acc: 0.8581
Epoch 4/5
40000/40000 [==============================] - 9s 219us/step - loss: 0.1883 - acc: 0.9307 - val_loss: 0.3610 - val_acc: 0.8490
Epoch 5/5
40000/40000 [==============================] - 9s 219us/step - loss: 0.1778 - acc: 0.9363 - val_loss: 0.3760 - val_acc: 0.8444


In [96]:
text = ["Keira Knightley does sublime justice to her real-life subject in this lithe, topical ode to female empowerment."]
text[0] = cleantext(text[0])
list_tokenized = tokenizer.texts_to_sequences(text)

maxlen = 200
X_test = pad_sequences(list_tokenized, maxlen=maxlen)
import numpy as np
y_pred = model.predict(X_test)


In [97]:
y_pred = np.rint(y_pred)
print(y_pred)

[[1.]]


In [25]:
from sklearn.metrics import accuracy_score
a = accuracy_score(y_test,y_pred)

In [26]:
print(a)

0.5067255571170448
